In [1]:

from __future__ import print_function
import os
import glob
from keras.models import Sequential
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, ReduceLROnPlateau, EarlyStopping
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv2D
from keras.layers.normalization import BatchNormalization
import numpy as np

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt

In [3]:
train_dir = "/home/asus/Desktop/Dataset_splits/train/"
test_dir = "/home/asus/Desktop/Dataset_splits/validation/"

In [4]:
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
early_stopper = EarlyStopping(min_delta=0.001, patience=10)
csv_logger = CSVLogger('editalexnet_scratch_log.csv')

In [5]:
def get_nb_files(directory):
    """Get number of files by searching directory recursively"""
    if not os.path.exists(directory):
        return 0
    cnt = 0
    for r, dirs, files in os.walk(directory):
        for dr in dirs:
            cnt += len(glob.glob(os.path.join(r, dr + "/*")))
    return cnt

In [6]:
def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'r.')
    plt.plot(epochs, val_acc, 'r')
    plt.title('Training and validation accuracy')

    plt.figure()
    plt.plot(epochs, loss, 'r.')
    plt.plot(epochs, val_loss, 'r-')
    plt.title('Training and validation loss')
    plt.show()

In [7]:
batch_size = 128
epochs = 25
nb_train_samples = get_nb_files(train_dir)
num_classes = len(glob.glob(train_dir + "/*"))
nb_val_samples = get_nb_files(test_dir)


In [8]:
# input image dimensions
IM_WIDTH, IM_HEIGHT = 150, 150
input_shape = (IM_WIDTH, IM_HEIGHT, 3)

In [9]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [10]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(IM_WIDTH, IM_HEIGHT), batch_size=batch_size)

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(IM_WIDTH, IM_HEIGHT), batch_size=batch_size)

Found 22110 images belonging to 33 classes.
Found 4132 images belonging to 33 classes.


In [11]:
model = Sequential()
model.add(Conv2D(64, (11, 11), input_shape=input_shape, padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (3, 3)))

model.add(Conv2D(128, (7, 7), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Convolution2D(192, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Convolution2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Flatten())
model.add(Dense(12 * 12 * 256, activation='relu'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4096, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes , activation='softmax'))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 150, 150, 64)      23296     
_________________________________________________________________
batch_normalization_1 (Batch (None, 150, 150, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 150, 150, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 128)       401536    
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 50, 128)       512       
_________________________________________________________________
activation_2 (Activation)    (None, 50, 50, 128)      

In [13]:
model.compile(optimizer='Adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
history_train = model.fit_generator(train_generator, nb_epoch=epochs, steps_per_epoch=nb_train_samples // batch_size,
                                    validation_data=test_generator, nb_val_samples=nb_val_samples // batch_size,
                                    class_weight='auto', callbacks=[lr_reducer, early_stopper, csv_logger])


/home/asus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/home/asus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., steps_per_epoch=172, validation_data=<keras.pre..., class_weight="auto", callbacks=[<keras.ca..., epochs=25, validation_steps=32)`
  after removing the cwd from sys.path.


Epoch 1/25
172/172 [==============================] - 2460s 14s/step - loss: 1.4997 - accuracy: 0.5993 - val_loss: 3.6040 - val_accuracy: 0.1069
Epoch 2/25
172/172 [==============================] - 2422s 14s/step - loss: 0.7655 - accuracy: 0.7576 - val_loss: 1.7228 - val_accuracy: 0.5659
Epoch 3/25
172/172 [==============================] - 2393s 14s/step - loss: 0.5392 - accuracy: 0.8238 - val_loss: 1.1815 - val_accuracy: 0.6656
Epoch 4/25
172/172 [==============================] - 2614s 15s/step - loss: 0.4756 - accuracy: 0.8504 - val_loss: 0.8117 - val_accuracy: 0.7522
Epoch 5/25
172/172 [==============================] - 2374s 14s/step - loss: 0.3371 - accuracy: 0.8843 - val_loss: 0.6536 - val_accuracy: 0.8214
Epoch 6/25
172/172 [==============================] - 2500s 15s/step - loss: 0.3007 - accuracy: 0.9007 - val_loss: 0.4677 - val_accuracy: 0.8789
Epoch 7/25
172/172 [==============================] - 2381s 14s/step - loss: 0.2536 - accuracy: 0.9147 - val_loss: 0.8239 - val_ac

In [14]:
model.save('finalAlex_Net_scratch.h5')